In [1]:
# import nba_api as api

# api.library.debug.debug.DEBUG = True

from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import playercareerstats, playerawards, playerprofilev2, homepagev2
from nba_api.stats.library.parameters import StatType

TIM_DUNCAN = 1495

player_select = TIM_DUNCAN

In [2]:
playercareerstats.

SyntaxError: invalid syntax (4240991527.py, line 1)

## Teams and Team ids

In [ ]:
t = teams._get_teams()

team_names = [(team['full_name'],team['id']) for team in t]
team_names

## Fetch All NBA Players ever

In [ ]:
# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
print("Number of players fetched: {}".format(len(nba_players)))
nba_players[2]

## Player Career Stats

In [ ]:
stats = playercareerstats.PlayerCareerStats(player_id=player_select,per_mode36="PerGame") # PerGame or Totals
stats.get_data_frames()[0].columns

## Player Awards (including "NBA Most Valuable Player")

In [ ]:
awards = playerawards.PlayerAwards(player_id=player_select).get_data_frames()
awards[0].columns

In [ ]:
pp = playerprofilev2.PlayerProfileV2(player_id=player_select)

In [ ]:
list(pp.get_data_frames()[0].columns)

In [ ]:

hp = homepagev2.HomePageV2()

In [ ]:
# hp.get_data_frames()

In [ ]:

stats2 = playercareerstats.PlayerCareerStats(player_id=player_select,per_mode36="PerGame",get_request=False) # PerGame or Totals
stats2.parameters["StatType"] = StatType.tracking
stats2.get_request()
list(stats2.get_data_frames()[0].columns)

In [3]:
import pandas as pd
import numpy as np

### Get All teams
from nba_api.stats.static import teams

nba_teams = teams.get_teams()

### Get games of current season
from nba_api.stats.endpoints import leaguegamefinder
games_per_team = []
for team in nba_teams:
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team['id'])
    games = gamefinder.get_data_frames()[0]
    games.GAME_DATE = pd.to_datetime(games.GAME_DATE)
    ### Here i choose games from the current 2023-2024 Season (without pre-season games)
    games_per_team.append(games[(games.SEASON_ID == '22023') & (games.GAME_DATE >= '2023-10-24')])

### Get the Offensive rating, the Defensive rating and other relative stats
from nba_api.stats.endpoints import boxscoreadvancedv3

target_stats = []
for team in games_per_team:
    ### Take number of Win and Losses
    WL = team.WL.value_counts().reset_index()
    W = WL[WL.WL == 'W']['count'].values[0]
    L = WL[WL.WL == 'L']['count'].values[0]
    ### Get classic stats
    target_stats.append({'team_abbreviation':team.TEAM_ABBREVIATION.unique()[0],
                         'team_name':team.TEAM_NAME.unique()[0],
                         'game_date':team.GAME_DATE.to_list(),
                         'W':W,
                         'L':L,
                         'offensive_rating':[], 
                         'defensive_rating':[],
                         'possessions':[] 
                         })
    
    ### Get advanced stats
    for id in team.GAME_ID.to_list():
        stats = boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=id) 
        ### If the team is an away team during the game, you take the stats in the awayTeam key
        if team[team.GAME_ID == id].MATCHUP.values[0][4] == "@":  ### Away Team
            target_stats[-1]['offensive_rating'].append(stats.get_dict()['boxScoreAdvanced']['awayTeam']['statistics']['offensiveRating'])
            target_stats[-1]['defensive_rating'].append(stats.get_dict()['boxScoreAdvanced']['awayTeam']['statistics']['defensiveRating'])
            target_stats[-1]['possessions'].append(stats.get_dict()['boxScoreAdvanced']['awayTeam']['statistics']['possessions'])
        else:
            target_stats[-1]['offensive_rating'].append(stats.get_dict()['boxScoreAdvanced']['homeTeam']['statistics']['offensiveRating'])
            target_stats[-1]['defensive_rating'].append(stats.get_dict()['boxScoreAdvanced']['homeTeam']['statistics']['defensiveRating'])
            target_stats[-1]['possessions'].append(stats.get_dict()['boxScoreAdvanced']['homeTeam']['statistics']['possessions'])

### Compute metrics
data = pd.DataFrame(target_stats)
data['GP'] = data.W + data.L
data['PCT_W'] = (data.W / data.GP)*100
data['PCT_W_rank'] = data.PCT_W.rank(method='min', ascending=False).astype('int64')
data.head()

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)